In [3]:
# pip install mysql-connector-python firebase-admin

In [6]:
# pip install sqlparse
# pip install pymongo

In [169]:
import re
from pymongo import MongoClient  # If using a MongoDB database
import sqlite3  # If using SQLite for initial SQL queries

In [ ]:
database_info = {
    "database_name": {
        "students": {
            "columns": {
                "StudentID": "INTEGER",
                "FirstName": "TEXT",
                "LastName": "TEXT",
                "Email": "TEXT"
            }
        },
        "enrollments": {
            "columns": {
                "EnrollmentID": "INTEGER",
                "StudentID": "INTEGER",
                "CourseID": "INTEGER",
                "Semester": "TEXT",
                "Grade": "TEXT"
            }
        },
        "courses": {
            "columns": {
                "CourseID": "INTEGER",
                "CourseName": "TEXT",
                "Instructor": "TEXT"
            }
        }
    }
}

In [591]:
import re
def parse_sql_to_mongo(sql_query, database_info):
    # Remove line breaks and extra spaces
    sql_query = ' '.join(sql_query.strip().split())

    # Initialize pipeline and collection name
    pipeline = []
    collection_name = ''
    aliases = {}  # Dictionary to track table aliases

    # Extract SELECT and FROM clauses
    select_match = re.search(r"SELECT\s+(.*?)\s+FROM\s+(\w+)(?:\s+AS\s+(\w+))?", sql_query, re.IGNORECASE)
    if select_match:
        select_clause = select_match.group(1)
        collection_name = select_match.group(2)
        main_alias = select_match.group(3) or collection_name
        aliases[main_alias] = collection_name
    else:
        raise ValueError("Invalid SQL query: Cannot find SELECT and FROM clauses.")

    # Helper function to clean field names based on their origin
    def clean_field(field):
        # Remove the alias if it's from the main table
        if field.startswith(f"{main_alias}.") or field.startswith(f"{collection_name}."):
            return field.split('.')[-1]
        return field

    # Helper function to remove alias from fields in aggregation expressions
    def clean_expression(expression):
        # Strip main table alias from any expression field reference
        return expression.replace(f"{main_alias}.", "").replace(f"{collection_name}.", "")

    # Handle the remaining parts of the query
    rest_of_query = sql_query[select_match.end():].strip()

    # Initialize variables for WHERE, GROUP BY, etc.
    match_conditions = {}
    project_fields = {'_id': 0}  # Exclude _id by default
    sort_fields = {}
    limit_value = None
    last_alias = main_alias  # Keep track of the last alias for join referencing
    aggregation_fields = {}
    having_conditions = {}
    join_count = 0

    join_pattern = re.compile(
        r"(INNER|LEFT|RIGHT|FULL\s+OUTER|FULL)?\s*JOIN\s+(\w+)(?:\s+AS\s+(\w+)|\s+(\w+))?\s+ON\s+(.+?)(?=\s*(INNER|LEFT|RIGHT|JOIN|FULL|WHERE|GROUP BY|ORDER BY|LIMIT|;|$))",
        re.IGNORECASE | re.DOTALL
    )

    while True:
        join_match = join_pattern.search(rest_of_query)
        if not join_match:
            break

        # Extract components of the JOIN clause
        join_type = (join_match.group(1) or '').strip().upper()
        table_name = join_match.group(2)
        alias = (join_match.group(3) or join_match.group(4) or table_name).strip()
        on_condition = join_match.group(5).strip()
        alias = alias or table_name
        aliases[alias] = table_name  # Store alias-to-table mapping

        # Parse the ON condition fields (e.g., "s.StudentID = e.StudentID")
        left_field, right_field = [f.strip() for f in on_condition.split('=')]
        local_field = clean_field(left_field)
        foreign_field = right_field.split('.')[-1]
        if join_count == 0:
            local_field = local_field.split('.')[-1]

        # Construct the MongoDB `$lookup` stage
        lookup_stage = {
            '$lookup': {
                'from': table_name,
                'localField': local_field,
                'foreignField': foreign_field,
                'as': alias
            }
        }
        pipeline.append(lookup_stage)

        # If INNER JOIN, add `$unwind` stage
        if join_type.upper() == 'INNER' or not join_type:
            pipeline.append({'$unwind': f'${alias}'})

        # Remove processed JOIN from the query
        rest_of_query = rest_of_query[:join_match.start()] + rest_of_query[join_match.end():].strip()
        join_count += 1

    # Extract WHERE clause
    where_match = re.search(r"WHERE\s+(.*?)(\s+GROUP BY|\s+ORDER BY|\s+HAVING|\s*+LIMIT|;|$)", rest_of_query, re.IGNORECASE)
    if where_match:
        where_clause = where_match.group(1)
        conditions = [cond.strip() for cond in re.split(r' AND | and ', where_clause)]
        for condition in conditions:
            match_cond = re.match(r'(\w+\.?\w*)\s*(=|>|<|>=|<=|!=)\s*(.+)', condition)
            if match_cond:
                field, operator, value = match_cond.groups()
                field = clean_field(field)  # Remove table prefix
                operator_map = {'=': '$eq', '>': '$gt', '<': '$lt', '>=': '$gte', '<=': '$lte', '!=': '$ne'}
                mongo_operator = operator_map.get(operator)
                value = value.strip()
                if value.startswith("'") and value.endswith("'"):
                    value = value[1:-1]
                elif value.startswith('"') and value.endswith('"'):
                    value = value[1:-1]
                else:
                    try:
                        value = float(value) if '.' in value else int(value)
                    except ValueError:
                        pass  # Keep as string
                match_conditions[field] = {mongo_operator: value}
            else:
                raise ValueError(f"Unable to parse WHERE condition: {condition}")

    # Add match conditions to pipeline
    if match_conditions:
        pipeline.append({'$match': match_conditions})

    # Parse SELECT clause
    for select_field in select_clause.split(','):
        select_field = select_field.strip()
        agg_match = re.match(r"(SUM|AVG|COUNT|MIN|MAX)\((.*?)\)\s+AS\s+(\w+)", select_field, re.IGNORECASE)
        if agg_match:
            # Aggregation function handling
            func, col, alias = agg_match.groups()
            col = clean_field(col)  # Clean the field within aggregation
            func_map = {'SUM': '$sum', 'AVG': '$avg', 'COUNT': '$sum', 'MIN': '$min', 'MAX': '$max'}
            mongo_func = func_map[func.upper()]
            aggregation_fields[alias] = {mongo_func: f"${col}" if func.upper() != 'COUNT' else 1}
            project_fields[alias] = f"${alias}"
        else:
            # Direct field selection handling
            clean_select_field = clean_field(select_field)
            clean_select_field_ = clean_select_field.split('.')[1] if '.' in clean_select_field else clean_select_field
            project_fields[clean_select_field_] = f"${clean_select_field}"

    # Extract GROUP BY clause
    group_by_match = re.search(r"GROUP BY\s+(.*?)(\s+HAVING|\s+ORDER BY|\s*+LIMIT|;|$)", rest_of_query, re.IGNORECASE)
    if group_by_match:
        group_by_fields = [clean_field(field.strip()) for field in group_by_match.group(1).split(',')]
        group_stage = {'$group': {'_id': {}}}
        for field in group_by_fields:
            if ' AS ' in field:
                field = field.split(' AS ')[1]
            group_stage['$group']['_id'][field] = f"${field}"
            project_fields[field] = f"${field}"

        # Process aggregation functions in SELECT clause
        for alias, agg_expr in aggregation_fields.items():
            group_stage['$group'][alias] = agg_expr

        pipeline.append(group_stage)

        # Extract HAVING clause
        having_match = re.search(r"HAVING\s+(.*?)(\s+ORDER BY|\s+LIMIT|;|$)", rest_of_query, re.IGNORECASE)
        if having_match:
            having_clause = having_match.group(1)
            having_conditions = []
            conditions = [cond.strip() for cond in re.split(r' AND | and ', having_clause)]
            for condition in conditions:
                match_cond = re.match(r'(\w+)\s*(=|>|<|>=|<=|!=)\s*(.+)', condition)
                if match_cond:
                    field, operator, value = match_cond.groups()
                    field = field.strip()
                    operator_map = {'=': '$eq', '>': '$gt', '<': '$lt', '>=': '$gte', '<=': '$lte', '!=': '$ne'}
                    mongo_operator = operator_map.get(operator)
                    value = value.strip()
                    if value.startswith("'") and value.endswith("'"):
                        value = value[1:-1]
                    elif value.startswith('"') and value.endswith('"'):
                        value = value[1:-1]
                    else:
                        try:
                            value = float(value) if '.' in value else int(value)
                        except ValueError:
                            pass  # Keep as string
                    having_conditions.append({field: {mongo_operator: value}})
                else:
                    raise ValueError(f"Unable to parse HAVING condition: {condition}")

            if having_conditions:
                # Combine multiple conditions using $and
                if len(having_conditions) > 1:
                    pipeline.append({'$match': {'$and': having_conditions}})
                else:
                    pipeline.append({'$match': having_conditions[0]})

        # Project fields after grouping and having
        pipeline.append({'$project': project_fields})

    else:
        # If no GROUP BY, just project the fields
        pipeline.append({'$project': project_fields})

    # Extract ORDER BY clause
    order_by_match = re.search(r"ORDER BY\s+(.*?)(ASC|DESC)?(\s+LIMIT|;|$)", rest_of_query, re.IGNORECASE)
    if order_by_match:
        order_by_fields = [clean_field(field.strip()) for field in order_by_match.group(1).split(',')]
        order_direction = 1
        direction = order_by_match.group(2)
        if direction and direction.upper() == 'DESC':
            order_direction = -1
        for field in order_by_fields:
            sort_fields[field] = order_direction

    # Add sort fields to pipeline
    if sort_fields:
        pipeline.append({'$sort': sort_fields})

    # Extract LIMIT clause
    limit_match = re.search(r"LIMIT\s+(\d+)", rest_of_query, re.IGNORECASE)
    if limit_match:
        limit_value = int(limit_match.group(1))
        pipeline.append({'$limit': limit_value})

    return collection_name, pipeline

In [592]:
def get_sqlite_data(db_path, sql_query):
    """
    Connects to SQLite and retrieves data based on the SQL query.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(sql_query)
    data = cursor.fetchall()
    conn.close()
    return data

In [615]:
sql_query = """
SELECT students.FirstName, students.LastName, courses.CourseName, enrollments.Grade
FROM students
INNER JOIN enrollments ON students.StudentID = enrollments.StudentID
INNER JOIN courses ON enrollments.CourseID = courses.CourseID
WHERE enrollments.Grade = 'A';
"""

In [616]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$lookup': {'from': 'enrollments', 'localField': 'StudentID', 'foreignField': 'StudentID', 'as': 'enrollments'}}, {'$unwind': '$enrollments'}, {'$lookup': {'from': 'courses', 'localField': 'enrollments.CourseID', 'foreignField': 'CourseID', 'as': 'courses'}}, {'$unwind': '$courses'}, {'$match': {'enrollments.Grade': {'$eq': 'A'}}}, {'$project': {'_id': 0, 'FirstName': '$FirstName', 'LastName': '$LastName', 'CourseName': '$courses.CourseName', 'Grade': '$enrollments.Grade'}}]
{'FirstName': 'John', 'LastName': 'Doe', 'CourseName': 'Data Structures', 'Grade': 'A'}
{'FirstName': 'John', 'LastName': 'Doe', 'CourseName': 'Database Systems', 'Grade': 'A'}
{'FirstName': 'Jim', 'LastName': 'Beam', 'CourseName': 'Data Structures', 'Grade': 'A'}
{'FirstName': 'John', 'LastName': 'Smith', 'CourseName': 'Data Structures', 'Grade': 'A'}


[('John', 'Doe', 'Data Structures', 'A'),
 ('John', 'Doe', 'Database Systems', 'A'),
 ('Jim', 'Beam', 'Data Structures', 'A'),
 ('John', 'Smith', 'Data Structures', 'A')]

In [595]:
sql_query = """
SELECT students.FirstName, students.LastName, enrollments.Grade
FROM students
INNER JOIN enrollments ON students.StudentID = enrollments.StudentID;
"""

In [596]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$lookup': {'from': 'enrollments', 'localField': 'StudentID', 'foreignField': 'StudentID', 'as': 'enrollments'}}, {'$unwind': '$enrollments'}, {'$project': {'_id': 0, 'FirstName': '$FirstName', 'LastName': '$LastName', 'Grade': '$enrollments.Grade'}}]
{'FirstName': 'John', 'LastName': 'Doe', 'Grade': 'A'}
{'FirstName': 'John', 'LastName': 'Doe', 'Grade': 'A'}
{'FirstName': 'Jane', 'LastName': 'Doe', 'Grade': 'B'}
{'FirstName': 'Jim', 'LastName': 'Beam', 'Grade': 'A'}
{'FirstName': 'Alice', 'LastName': 'Johnson', 'Grade': 'C'}
{'FirstName': 'John', 'LastName': 'Smith', 'Grade': 'A'}
{'FirstName': 'Bill', 'LastName': 'Chu', 'Grade': 'B'}


[('John', 'Doe', 'A'),
 ('John', 'Doe', 'A'),
 ('Jane', 'Doe', 'B'),
 ('Jim', 'Beam', 'A'),
 ('Alice', 'Johnson', 'C'),
 ('John', 'Smith', 'A'),
 ('Bill', 'Chu', 'B')]

In [597]:
sql_query = """
SELECT courses.CourseName, COUNT(enrollments.EnrollmentID) AS TotalEnrollments
FROM courses
INNER JOIN enrollments ON courses.CourseID = enrollments.CourseID
GROUP BY courses.CourseName;
"""

In [598]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$lookup': {'from': 'enrollments', 'localField': 'CourseID', 'foreignField': 'CourseID', 'as': 'enrollments'}}, {'$unwind': '$enrollments'}, {'$group': {'_id': {'CourseName': '$CourseName'}, 'TotalEnrollments': {'$sum': 1}}}, {'$project': {'_id': 0, 'CourseName': '$CourseName', 'TotalEnrollments': '$TotalEnrollments'}}]
{'TotalEnrollments': 1}
{'TotalEnrollments': 1}
{'TotalEnrollments': 3}
{'TotalEnrollments': 2}


[('Calculus', 2),
 ('Data Structures', 3),
 ('Database Systems', 1),
 ('Linear Algebra', 1)]

In [599]:
# SELECT

sql_query = """
SELECT StudentID FROM students;
"""

In [600]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$project': {'_id': 0, 'StudentID': '$StudentID'}}]
{'StudentID': 1}
{'StudentID': 2}
{'StudentID': 3}
{'StudentID': 4}
{'StudentID': 5}
{'StudentID': 6}
{'StudentID': 7}


[(1,), (2,), (3,), (4,), (5,), (6,), (7,)]

In [601]:
# WHERE

sql_query = """
SELECT s.FirstName, s.LastName
FROM students AS s
JOIN enrollments e ON s.StudentID = e.StudentID
WHERE e.CourseID = 101;
"""

In [602]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$lookup': {'from': 'enrollments', 'localField': 'StudentID', 'foreignField': 'StudentID', 'as': 'e'}}, {'$unwind': '$e'}, {'$match': {'e.CourseID': {'$eq': 101}}}, {'$project': {'_id': 0, 'FirstName': '$FirstName', 'LastName': '$LastName'}}]
{'FirstName': 'John', 'LastName': 'Doe'}
{'FirstName': 'Jim', 'LastName': 'Beam'}
{'FirstName': 'John', 'LastName': 'Smith'}


[('John', 'Doe'), ('Jim', 'Beam'), ('John', 'Smith')]

In [603]:
# ORDER BY

sql_query = """
SELECT FirstName, LastName
FROM students
ORDER BY LastName ASC;
"""

In [604]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$project': {'_id': 0, 'FirstName': '$FirstName', 'LastName': '$LastName'}}, {'$sort': {'LastName': 1}}]
{'FirstName': 'Jim', 'LastName': 'Beam'}
{'FirstName': 'Bill', 'LastName': 'Chu'}
{'FirstName': 'John', 'LastName': 'Doe'}
{'FirstName': 'Jane', 'LastName': 'Doe'}
{'FirstName': 'Alice', 'LastName': 'Johnson'}
{'FirstName': 'David', 'LastName': 'Small'}
{'FirstName': 'John', 'LastName': 'Smith'}


[('Jim', 'Beam'),
 ('Bill', 'Chu'),
 ('John', 'Doe'),
 ('Jane', 'Doe'),
 ('Alice', 'Johnson'),
 ('David', 'Small'),
 ('John', 'Smith')]

In [605]:
# JOIN

sql_query = """
SELECT s.FirstName, s.LastName, c.CourseName
FROM students AS s
JOIN enrollments e ON s.StudentID = e.StudentID
JOIN courses c ON e.CourseID = c.CourseID;
"""

In [606]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$lookup': {'from': 'enrollments', 'localField': 'StudentID', 'foreignField': 'StudentID', 'as': 'e'}}, {'$unwind': '$e'}, {'$lookup': {'from': 'courses', 'localField': 'e.CourseID', 'foreignField': 'CourseID', 'as': 'c'}}, {'$unwind': '$c'}, {'$project': {'_id': 0, 'FirstName': '$FirstName', 'LastName': '$LastName', 'CourseName': '$c.CourseName'}}]
{'FirstName': 'John', 'LastName': 'Doe', 'CourseName': 'Data Structures'}
{'FirstName': 'John', 'LastName': 'Doe', 'CourseName': 'Database Systems'}
{'FirstName': 'Jane', 'LastName': 'Doe', 'CourseName': 'Calculus'}
{'FirstName': 'Jim', 'LastName': 'Beam', 'CourseName': 'Data Structures'}
{'FirstName': 'Alice', 'LastName': 'Johnson', 'CourseName': 'Linear Algebra'}
{'FirstName': 'John', 'LastName': 'Smith', 'CourseName': 'Data Structures'}
{'FirstName': 'Bill', 'LastName': 'Chu', 'CourseName': 'Calculus'}


[('John', 'Doe', 'Data Structures'),
 ('John', 'Doe', 'Database Systems'),
 ('Jane', 'Doe', 'Calculus'),
 ('Jim', 'Beam', 'Data Structures'),
 ('Alice', 'Johnson', 'Linear Algebra'),
 ('John', 'Smith', 'Data Structures'),
 ('Bill', 'Chu', 'Calculus')]

In [607]:
# GROUP BY

sql_query = """
SELECT c.CourseName, COUNT(e.StudentID) AS StudentCount
FROM courses AS c
JOIN enrollments e ON c.CourseID = e.CourseID
GROUP BY c.CourseName;
"""

In [608]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$lookup': {'from': 'enrollments', 'localField': 'CourseID', 'foreignField': 'CourseID', 'as': 'e'}}, {'$unwind': '$e'}, {'$group': {'_id': {'CourseName': '$CourseName'}, 'StudentCount': {'$sum': 1}}}, {'$project': {'_id': 0, 'CourseName': '$CourseName', 'StudentCount': '$StudentCount'}}]
{'StudentCount': 1}
{'StudentCount': 3}
{'StudentCount': 2}
{'StudentCount': 1}


[('Calculus', 2),
 ('Data Structures', 3),
 ('Database Systems', 1),
 ('Linear Algebra', 1)]

In [609]:
# HAVING

sql_query = """
SELECT c.CourseName, COUNT(e.StudentID) AS StudentCount
FROM courses AS c
JOIN enrollments e ON c.CourseID = e.CourseID
GROUP BY c.CourseName
HAVING StudentCount > 2;
"""

In [610]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$lookup': {'from': 'enrollments', 'localField': 'CourseID', 'foreignField': 'CourseID', 'as': 'e'}}, {'$unwind': '$e'}, {'$group': {'_id': {'CourseName': '$CourseName'}, 'StudentCount': {'$sum': 1}}}, {'$match': {'StudentCount': {'$gt': 2}}}, {'$project': {'_id': 0, 'CourseName': '$CourseName', 'StudentCount': '$StudentCount'}}]
{'StudentCount': 3}


[('Data Structures', 3)]

In [611]:
# LIMIT

sql_query = """
SELECT StudentID, FirstName, LastName FROM students
LIMIT 5;
"""

In [612]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$project': {'_id': 0, 'StudentID': '$StudentID', 'FirstName': '$FirstName', 'LastName': '$LastName'}}, {'$limit': 5}]
{'StudentID': 1, 'FirstName': 'John', 'LastName': 'Doe'}
{'StudentID': 2, 'FirstName': 'Jane', 'LastName': 'Doe'}
{'StudentID': 3, 'FirstName': 'Jim', 'LastName': 'Beam'}
{'StudentID': 4, 'FirstName': 'Alice', 'LastName': 'Johnson'}
{'StudentID': 5, 'FirstName': 'John', 'LastName': 'Smith'}


[(1, 'John', 'Doe'),
 (2, 'Jane', 'Doe'),
 (3, 'Jim', 'Beam'),
 (4, 'Alice', 'Johnson'),
 (5, 'John', 'Smith')]

In [621]:
# AVG

# sql_query = """
# SELECT AVG(Grade) AS AverageGrade
# FROM enrollments
# WHERE CourseID = 101
# GROUP BY CourseID;
# """

In [620]:
collection_name, pipeline = parse_sql_to_mongo(sql_query, database_info)
print(pipeline)
client = MongoClient('mongodb://localhost:27017/')
db = client['test_db']
collection = db[collection_name]

results = collection.aggregate(pipeline)
for doc in results:
    print(doc)
get_sqlite_data("college_data.db", sql_query)

[{'$match': {'CourseID': {'$eq': 101}}}, {'$group': {'_id': {'CourseID': '$CourseID'}, 'AverageGrade': {'$avg': '$Grade'}}}, {'$project': {'_id': 0, 'AverageGrade': '$AverageGrade', 'CourseID': '$CourseID'}}]
{'AverageGrade': None}


[(0.0,)]

In [24]:
# import sqlite3
# import pandas as pd

# # Load the CSV data into a DataFrame
# file_path = 'coffee_shop_sales.csv'
# data = pd.read_csv(file_path)

# # Specify the path for the SQLite database
# db_path = 'coffee_shop_sales.db'

# # Connect to the SQLite database (or create it if it doesn't exist)
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Define the table structure to match the CSV data
# create_table_query = '''
# CREATE TABLE IF NOT EXISTS coffee_shop_sales (
#     transaction_id INTEGER,
#     transaction_date TEXT,
#     transaction_time TEXT,
#     transaction_qty INTEGER,
#     store_id INTEGER,
#     store_location TEXT,
#     product_id INTEGER,
#     unit_price REAL,
#     product_category TEXT,
#     product_type TEXT,
#     product_detail TEXT
# );
# '''
# cursor.execute(create_table_query)

# # Insert the CSV data into the SQLite table
# data.to_sql('coffee_shop_sales', conn, if_exists='replace', index=False)

# # Commit changes and close the connection
# conn.commit()
# conn.close()

# print("Database created and data added successfully.")


In [211]:
# # Importing data from each CSV into a SQLite database and setting up corresponding tables
# import sqlite3
# # Let's load and examine the structure of the newly uploaded files to understand the data and generate appropriate SQL queries

# # Load the CSV files
# students_df = pd.read_csv('students.csv')
# enrollments_df = pd.read_csv('enrollments.csv')
# courses_df = pd.read_csv('courses.csv')

# # Display the first few rows of each dataframe to understand the data structure
# students_df.head(), enrollments_df.head(), courses_df.head()

# # Database path
# db_path = 'college_data.db'

# # Connect to SQLite database (it will create the database if it doesn't exist)
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Create tables for each CSV file based on the data structure we observed
# # Table for students
# create_students_table = '''
# CREATE TABLE IF NOT EXISTS students (
#     StudentID INTEGER PRIMARY KEY,
#     FirstName TEXT,
#     LastName TEXT,
#     Email TEXT,
#     Major TEXT,
#     AdvisorID INTEGER,
#     AdvisorName TEXT
# );
# '''
# cursor.execute(create_students_table)

# # Table for enrollments
# create_enrollments_table = '''
# CREATE TABLE IF NOT EXISTS enrollments (
#     EnrollmentID INTEGER PRIMARY KEY,
#     StudentID INTEGER,
#     CourseID INTEGER,
#     Semester TEXT,
#     Grade TEXT,
#     FOREIGN KEY (StudentID) REFERENCES students(StudentID),
#     FOREIGN KEY (CourseID) REFERENCES courses(CourseID)
# );
# '''
# cursor.execute(create_enrollments_table)

# # Table for courses
# create_courses_table = '''
# CREATE TABLE IF NOT EXISTS courses (
#     CourseID INTEGER PRIMARY KEY,
#     CourseName TEXT,
#     InstructorID INTEGER,
#     InstructorName TEXT,
#     CreditHours INTEGER
# );
# '''
# cursor.execute(create_courses_table)

# # Insert data from DataFrames into SQLite tables
# students_df.to_sql('students', conn, if_exists='replace', index=False)
# enrollments_df.to_sql('enrollments', conn, if_exists='replace', index=False)
# courses_df.to_sql('courses', conn, if_exists='replace', index=False)

# # Commit changes and close the connection
# conn.commit()
# conn.close()

# "Data has been successfully imported into the SQLite database 'college_data.db'."


"Data has been successfully imported into the SQLite database 'college_data.db'."